## Import Libraries

In [40]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 

## Import Data

In [41]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

In [42]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, random_state = 42, shuffle = True)
X_train.shape, Y_train.shape

((19200, 30), (19200,))

## SMOTE

In [43]:
smote = SMOTENC(list(range(0,18)), sampling_strategy='auto', k_neighbors=10, n_jobs=1)
X_train,Y_train =  smote.fit_resample(X_train, Y_train)

pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
             ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
X_train.shape, Y_train.shape

C:\Users\Janith\.conda\envs\mlbasic\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Janith\.conda\envs\mlbasic\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


((29850, 30), (29850,))

## Normalizing

In [25]:
# Standard Normalizing Data
sc = StandardScaler()
sc.fit(X_tot)
X_tot = sc.fit_transform(X_tot)
X_train = sc.transform(X_train)
X_cross = sc.transform(X_cross)
test_data = sc.fit_transform(test_data)

## NN Model

In [44]:
model = Sequential()
model.add(Dense(16, input_dim=30, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100, batch_size=512, validation_data=(X_cross, Y_cross))

Train on 29850 samples, validate on 4800 samples
Epoch 1/100
29850/29850 [==============================] - 1s 41us/sample - loss: 4291.6866 - acc: 0.5363 - val_loss: 1053.5260 - val_acc: 0.6271
Epoch 2/100
29850/29850 [==============================] - 0s 15us/sample - loss: 773.0442 - acc: 0.5596 - val_loss: 528.8763 - val_acc: 0.6579
Epoch 3/100
29850/29850 [==============================] - 0s 15us/sample - loss: 476.7939 - acc: 0.5521 - val_loss: 426.3454 - val_acc: 0.6131
Epoch 4/100
29850/29850 [==============================] - 0s 15us/sample - loss: 338.6508 - acc: 0.5474 - val_loss: 305.9949 - val_acc: 0.6167
Epoch 5/100
29850/29850 [==============================] - 0s 15us/sample - loss: 250.7550 - acc: 0.5417 - val_loss: 239.8536 - val_acc: 0.6100
Epoch 6/100
29850/29850 [==============================] - 0s 15us/sample - loss: 195.3644 - acc: 0.5424 - val_loss: 183.2099 - val_acc: 0.6413
Epoch 7/100
29850/29850 [==============================] - 0s 15us/sample - loss: 150

Epoch 58/100
29850/29850 [==============================] - 0s 16us/sample - loss: 6.5907 - acc: 0.5564 - val_loss: 42.4380 - val_acc: 0.2433
Epoch 59/100
29850/29850 [==============================] - 0s 16us/sample - loss: 13.2384 - acc: 0.5485 - val_loss: 20.6958 - val_acc: 0.3200
Epoch 60/100
29850/29850 [==============================] - 0s 15us/sample - loss: 7.1985 - acc: 0.5575 - val_loss: 7.7051 - val_acc: 0.7521
Epoch 61/100
29850/29850 [==============================] - 0s 15us/sample - loss: 8.5330 - acc: 0.5571 - val_loss: 5.0219 - val_acc: 0.7477
Epoch 62/100
29850/29850 [==============================] - 1s 21us/sample - loss: 7.8062 - acc: 0.5607 - val_loss: 3.6595 - val_acc: 0.4552
Epoch 63/100
29850/29850 [==============================] - 1s 17us/sample - loss: 6.6955 - acc: 0.5572 - val_loss: 13.4077 - val_acc: 0.3265
Epoch 64/100
29850/29850 [==============================] - 0s 17us/sample - loss: 5.3869 - acc: 0.5592 - val_loss: 3.1488 - val_acc: 0.5767
Epoch 65/

## Prediction and Evaluation

In [32]:
def predict_y(X_set):
    threshold = 0.5
    return np.where(model.predict(X_set)>= threshold, 1, 0)

predict_train = predict_y(X_train)
predict_cross = predict_y(X_cross)
predict_tot = predict_y(X_tot)
predict_test = predict_y(test_data)

model.predict(X_train).shape

(19200, 1)

In [33]:
# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 82.396%       AUC: 0.6611
Cross Validation Accuracy : 82.125%       AUC: 0.6523 
Total Set Accuracy        : 82.342%       AUC: 0.6593
Confusion Matrix for Cross Validation Set :
 [[3572  173]
 [ 685  370]]
Number of Ones Predicted for Test Data : 689
              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89      3745
         1.0       0.68      0.35      0.46      1055

    accuracy                           0.82      4800
   macro avg       0.76      0.65      0.68      4800
weighted avg       0.80      0.82      0.80      4800



In [34]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/SubmissionNN.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')